# Лабораторна робота № 2. ІП-14 Бабіч Денис

### Імпортування модулів

In [43]:
import cv2
import numpy

### 1. Використовуючи будь-яку фотографію з декількома людьми, виявити на ньому обличчя, очi, усмiшку. Порахувати кiлькiсть осiб на фото.

In [48]:
_SCALING_FACTOR = 0.25

eye_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_eye.xml")
smile_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_smile.xml")
face_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_frontalface_default.xml")

image = cv2.imread("Data/1.jpg")
image = cv2.resize(image, None, fx = _SCALING_FACTOR, fy = _SCALING_FACTOR, interpolation = cv2.INTER_AREA)

gray_filter = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

faces = face_classifier.detectMultiScale(image, scaleFactor = 1.05, minNeighbors = 15)

print(f"Було знайдено {len(faces)} облич/обличчя!")

for (x, y, width, height) in faces:
    roi_color = image[y:(y + height), x:(x + width)]
    roi_gray = gray_filter[y:(y + height), x:(x + width)]

    cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)

    eyes = eye_classifier.detectMultiScale(roi_gray, scaleFactor = 1.1, minNeighbors = 5)
    smiles = smile_classifier.detectMultiScale(roi_gray, scaleFactor = 1.15, minNeighbors = 17)

    for (x, y, width, height) in eyes:
       cv2.rectangle(roi_color, (x, y), (x + width, y + height), (255, 0, 0), 2)

    for (x, y, width, height) in smiles:
       cv2.rectangle(roi_color, (x, y), (x + width, y + height), (0, 0, 255), 2)

cv2.imshow("Detected face features", image)

cv2.waitKey(0)
cv2.destroyAllWindows()

Було знайдено 6 облич/обличчя!


### 2. Зробити розпізнавання використовуючи будь-яке відео з обличчям людини, тривалiстю не менше 30 секунд. Можна використати камеру ноутбука.

In [25]:
_KEY_ESCAPE = '\x1b'

eye_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_eye.xml")
smile_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_smile.xml")
face_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_frontalface_default.xml")

cv2.startWindowThread()

capture = cv2.VideoCapture("Data/2.mp4")

while True:
    _, frame = capture.read()

    if frame is None:
        break

    gray_filter = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_classifier.detectMultiScale(gray_filter, scaleFactor = 1.05, minNeighbors = 5)

    for (x, y, width, height) in faces:
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)

        roi_color = frame[y:(y + height), x:(x + width)]
        roi_gray = gray_filter[y:(y + height), x:(x + width)]

        eyes = eye_classifier.detectMultiScale(roi_gray, scaleFactor = 1.1, minNeighbors = 5)

        for (x, y, width, height) in eyes:
            cv2.rectangle(roi_color, (x, y), (x + width, y + height), (255, 0, 0), 2)

        smiles = smile_classifier.detectMultiScale(roi_gray, scaleFactor = 1.2, minNeighbors = 125)

        for (x, y, width, height) in smiles:
            cv2.rectangle(roi_color, (x, y), (x + width, y + height), (0, 0, 255), 2)

    cv2.imshow("Detected individuals", frame)

    if cv2.waitKey(1) & 0XFF == ord(_KEY_ESCAPE):
        break

capture.release()
cv2.destroyAllWindows()

### 3. Обробити вiдеофал, так щоб вiн видiляв пiшоходiв i, по можливостi, їхнi обличчя. Файл можна взяти з youtube i вирiзати ролик тривалiстю не менше 30 секунд.

In [41]:
_KEY_ESCAPE = '\x1b'

hog_descriptor = cv2.HOGDescriptor()
hog_descriptor.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

eye_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_eye.xml")
smile_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_smile.xml")
face_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_frontalface_default.xml")

cv2.startWindowThread()

capture = cv2.VideoCapture("Data/3.mp4")

while True:
    _, frame = capture.read()

    if frame is None:
        break

    gray_filter = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    people = hog_descriptor.detectMultiScale(gray_filter, winStride = (8, 8), padding = (16, 16), scale = 1.25)
    people = numpy.array([[human_x, human_y, human_x + human_w, human_y + human_h] for (human_x, human_y, human_w, human_h) in people[0]])

    for (x, y, width, height) in people:
        roi_color = frame[y:(y + height), x:(x + width)]
        roi_gray = gray_filter[y:(y + height), x:(x + width)]

        cv2.rectangle(frame, (x, y), (width, height), (0, 255, 255), 2)

        faces = face_classifier.detectMultiScale(roi_gray, scaleFactor = 1.05, minNeighbors = 5)

        for (x, y, width, height) in faces:
            cv2.rectangle(roi_color, (x, y), (x + width, y + height), (0, 255, 0), 2)
    
    cv2.imshow("Detected individuals", frame)

    if cv2.waitKey(1) & 0XFF == ord(_KEY_ESCAPE):
        break

capture.release()
cv2.destroyAllWindows()